In [3]:
import pymysql

db_info = {
    "host":"192.168.1.80",
    "user":"admin",
    "password":"123",
    "database":"management_systems",
    "charset":"utf8"
}

conn = pymysql.connect(**db_info)
cursor = conn.cursor()

In [4]:
## 上节课我们查询数据是这样做的
sql = "select * from school"
cursor.execute(sql)
# 取结果
result = cursor.fetchall()
result

((1, '第一实验小学', '南京路1号'),
 (2, '第十中学', '北京路2号'),
 (3, '实验中学', ''),
 (4, '第八中学', None),
 (6, '第六中学', None))

In [5]:
type(result)

tuple

In [7]:
## 这节课我们这样做
import pandas as pd
result = pd.read_sql(sql,conn)
result

,id,school_name,school_address
0,1,第一实验小学,南京路1号
1,2,第十中学,北京路2号
2,3,实验中学,
3,4,第八中学,None
4,6,第六中学,None


In [9]:
type(result)

pandas.core.frame.DataFrame

## 关联查询和dataframe数据类型中的数据关联对比

In [12]:

get_school_data_sql = 'select * from school'
get_grade_sql = 'select * from grade'
school_data = pd.read_sql(get_school_data_sql,conn)
grade_data = pd.read_sql(get_grade_sql,conn)

In [13]:
school_data

,id,school_name,school_address
0,1,第一实验小学,南京路1号
1,2,第十中学,北京路2号
2,3,实验中学,
3,4,第八中学,None
4,6,第六中学,None


In [14]:
grade_data

,id,school_id,grade_name
0,1,1,一年级
1,2,1,二年级
2,3,1,三年级
3,4,1,四年级
4,5,1,五年级
5,6,1,六年级
6,7,2,一年级
7,8,2,二年级
8,9,2,三年级
9,10,3,一年级


In [15]:
## 我们使用pandas中的merge来进行数据的关联
df = pd.merge(
    school_data, # 左表
    grade_data,  # 右表
    left_on = ['id'], # 左边的表取出哪个字段和右边的表进行关联
    right_on = ['school_id'], # 右边的表取出哪个字段和左边的表关联
    how = 'left'  # 这个就是用什么关联方式
)
df

,id_x,school_name,school_address,id_y,school_id,grade_name
0,1,第一实验小学,南京路1号,1.0,1.0,一年级
1,1,第一实验小学,南京路1号,2.0,1.0,二年级
2,1,第一实验小学,南京路1号,3.0,1.0,三年级
3,1,第一实验小学,南京路1号,4.0,1.0,四年级
4,1,第一实验小学,南京路1号,5.0,1.0,五年级
5,1,第一实验小学,南京路1号,6.0,1.0,六年级
6,2,第十中学,北京路2号,7.0,2.0,一年级
7,2,第十中学,北京路2号,8.0,2.0,二年级
8,2,第十中学,北京路2号,9.0,2.0,三年级
9,3,实验中学,,10.0,3.0,一年级


In [18]:
left_join_sql = """
select s.id sid,s.school_name , s.school_address, g.*
from school s
left join grade g
on s.id = g.school_id
"""

result = pd.read_sql(left_join_sql,conn)
result

,sid,school_name,school_address,id,school_id,grade_name
0,1,第一实验小学,南京路1号,1.0,1.0,一年级
1,1,第一实验小学,南京路1号,2.0,1.0,二年级
2,1,第一实验小学,南京路1号,3.0,1.0,三年级
3,1,第一实验小学,南京路1号,4.0,1.0,四年级
4,1,第一实验小学,南京路1号,5.0,1.0,五年级
5,1,第一实验小学,南京路1号,6.0,1.0,六年级
6,2,第十中学,北京路2号,7.0,2.0,一年级
7,2,第十中学,北京路2号,8.0,2.0,二年级
8,2,第十中学,北京路2号,9.0,2.0,三年级
9,3,实验中学,,10.0,3.0,一年级


In [19]:
## 多表之间的关联
left_join_sql = """
select s.*, g.*,c.*
from school s
left join grade g
on s.id = g.school_id
left join class c
on g.id = c.grade_id

"""
result = pd.read_sql(left_join_sql,conn)
result

,id,school_name,school_address,id,school_id,grade_name,id,grade_id,class_name
0,1,第一实验小学,南京路1号,1.0,1.0,一年级,1.0,1.0,一班
1,1,第一实验小学,南京路1号,1.0,1.0,一年级,2.0,1.0,二班
2,1,第一实验小学,南京路1号,1.0,1.0,一年级,3.0,1.0,三班
3,1,第一实验小学,南京路1号,2.0,1.0,二年级,4.0,2.0,一班
4,1,第一实验小学,南京路1号,2.0,1.0,二年级,5.0,2.0,二班
5,1,第一实验小学,南京路1号,2.0,1.0,二年级,6.0,2.0,三班
6,1,第一实验小学,南京路1号,3.0,1.0,三年级,7.0,3.0,一班
7,1,第一实验小学,南京路1号,3.0,1.0,三年级,8.0,3.0,二班
8,1,第一实验小学,南京路1号,3.0,1.0,三年级,9.0,3.0,三班
9,1,第一实验小学,南京路1号,4.0,1.0,四年级,10.0,4.0,一班


In [20]:
df = pd.merge(
    school_data, # 左表
    grade_data,  # 右表
    left_on = ['id'], # 左边的表取出哪个字段和右边的表进行关联
    right_on = ['school_id'], # 右边的表取出哪个字段和左边的表关联
    how = 'left'  # 这个就是用什么关联方式
)
df

,id_x,school_name,school_address,id_y,school_id,grade_name
0,1,第一实验小学,南京路1号,1.0,1.0,一年级
1,1,第一实验小学,南京路1号,2.0,1.0,二年级
2,1,第一实验小学,南京路1号,3.0,1.0,三年级
3,1,第一实验小学,南京路1号,4.0,1.0,四年级
4,1,第一实验小学,南京路1号,5.0,1.0,五年级
5,1,第一实验小学,南京路1号,6.0,1.0,六年级
6,2,第十中学,北京路2号,7.0,2.0,一年级
7,2,第十中学,北京路2号,8.0,2.0,二年级
8,2,第十中学,北京路2号,9.0,2.0,三年级
9,3,实验中学,,10.0,3.0,一年级


In [22]:
class_data = pd.read_sql('select * from class',conn)
df2 = pd.merge(
    df, # 左表
    class_data,  # 右表
    left_on = ['id_y'], # 左边的表取出哪个字段和右边的表进行关联
    right_on = ['grade_id'], # 右边的表取出哪个字段和左边的表关联
    how = 'left'  # 这个就是用什么关联方式
)
df2

,id_x,school_name,school_address,id_y,school_id,grade_name,id,grade_id,class_name
0,1,第一实验小学,南京路1号,1.0,1.0,一年级,1.0,1.0,一班
1,1,第一实验小学,南京路1号,1.0,1.0,一年级,2.0,1.0,二班
2,1,第一实验小学,南京路1号,1.0,1.0,一年级,3.0,1.0,三班
3,1,第一实验小学,南京路1号,2.0,1.0,二年级,4.0,2.0,一班
4,1,第一实验小学,南京路1号,2.0,1.0,二年级,5.0,2.0,二班
5,1,第一实验小学,南京路1号,2.0,1.0,二年级,6.0,2.0,三班
6,1,第一实验小学,南京路1号,3.0,1.0,三年级,7.0,3.0,一班
7,1,第一实验小学,南京路1号,3.0,1.0,三年级,8.0,3.0,二班
8,1,第一实验小学,南京路1号,3.0,1.0,三年级,9.0,3.0,三班
9,1,第一实验小学,南京路1号,4.0,1.0,四年级,10.0,4.0,一班


In [23]:
## 右关联 和 全关联
df = pd.merge(
    school_data, # 左表
    grade_data,  # 右表
    left_on = ['id'], # 左边的表取出哪个字段和右边的表进行关联
    right_on = ['school_id'], # 右边的表取出哪个字段和左边的表关联
    how = 'right'  # 这个就是用什么关联方式
)
df

,id_x,school_name,school_address,id_y,school_id,grade_name
0,1.0,第一实验小学,南京路1号,1,1,一年级
1,1.0,第一实验小学,南京路1号,2,1,二年级
2,1.0,第一实验小学,南京路1号,3,1,三年级
3,1.0,第一实验小学,南京路1号,4,1,四年级
4,1.0,第一实验小学,南京路1号,5,1,五年级
5,1.0,第一实验小学,南京路1号,6,1,六年级
6,2.0,第十中学,北京路2号,7,2,一年级
7,2.0,第十中学,北京路2号,8,2,二年级
8,2.0,第十中学,北京路2号,9,2,三年级
9,3.0,实验中学,,10,3,一年级


In [24]:
df = pd.merge(
    school_data, # 左表
    grade_data,  # 右表
    left_on = ['id'], # 左边的表取出哪个字段和右边的表进行关联
    right_on = ['school_id'], # 右边的表取出哪个字段和左边的表关联
    how = 'inner'  # 这个就是用什么关联方式
)
df

,id_x,school_name,school_address,id_y,school_id,grade_name
0,1,第一实验小学,南京路1号,1,1,一年级
1,1,第一实验小学,南京路1号,2,1,二年级
2,1,第一实验小学,南京路1号,3,1,三年级
3,1,第一实验小学,南京路1号,4,1,四年级
4,1,第一实验小学,南京路1号,5,1,五年级
5,1,第一实验小学,南京路1号,6,1,六年级
6,2,第十中学,北京路2号,7,2,一年级
7,2,第十中学,北京路2号,8,2,二年级
8,2,第十中学,北京路2号,9,2,三年级
9,3,实验中学,,10,3,一年级
